# 📈 Double Sort Investment Strategy: EPS/P and Debt/EBITDA
BAFI508 – Data-driven Investments
Modified sample code for proposal implementation

In [ ]:
# ✅ Step 1: Load and prepare CRSP
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import statsmodels.api as sm

path_data = '/content/drive/MyDrive/BAFI508/'
crsp = pd.read_csv(path_data + 'CRSP_Monthly_2018.csv')
crsp.columns = map(str.lower, crsp.columns)

crsp['ret'] = pd.to_numeric(crsp['ret'], errors='coerce')
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y%m%d')
crsp['year'] = crsp['date'].dt.year
crsp['month'] = crsp['date'].dt.month
crsp['mktcap'] = crsp['shrout'] * crsp['prc'].abs()

crsp = crsp[crsp['shrcd'].isin([10, 11])]
crsp = crsp[crsp['exchcd'].isin([1, 2, 3])]
crsp = crsp.drop_duplicates(subset=['date', 'permno'])

In [ ]:
# ✅ Step 2: Load and prepare Compustat (CCM)
ccm = pd.read_csv(path_data + 'Compustat_Annual_2018.csv')
ccm.columns = map(str.lower, ccm.columns)

# Merge CRSP price to compute EPS/P
crsp_prices = crsp[['permno', 'year', 'prc']].copy()
crsp_prices = crsp_prices.groupby(['permno', 'year']).last().reset_index()
crsp_prices.rename(columns={'permno': 'lpermno'}, inplace=True)
ccm = ccm.merge(crsp_prices, how='left', on=['lpermno', 'year'])

# Compute EPS/P using Net Income and Common Shares Outstanding
ccm['eps_p'] = (ccm['ni'] / ccm['csho']) / ccm['prc']

# Compute Debt/EBITDA
ccm['debt_ebitda'] = (ccm['dltt'].fillna(0) + ccm['dlc'].fillna(0)) / ccm['ebitda']

# Drop rows with missing key variables
ccm = ccm.dropna(subset=['eps_p', 'debt_ebitda'])

In [ ]:
# ✅ Step 3: Double Sort and Portfolio Construction
results = []
for year in tqdm(range(1981, 2018)):
    prior_year = year - 1
    eps_data = ccm[ccm['year'] == prior_year].copy()
    eps_data = eps_data[['lpermno', 'eps_p', 'debt_ebitda']]
    eps_data = eps_data.dropna()

    eps_data['eps_rank'] = pd.qcut(eps_data['eps_p'], 5, labels=False)

    merged_data = crsp[(crsp['year'] == year)].merge(eps_data, left_on='permno', right_on='lpermno')

    for eps_rank in range(5):
        temp = merged_data[merged_data['eps_rank'] == eps_rank].copy()
        temp['lev_rank'] = pd.qcut(temp['debt_ebitda'], 5, labels=False)
        for lev_rank in range(5):
            group = temp[temp['lev_rank'] == lev_rank]
            port_ret = group.groupby('date')['ret'].mean().reset_index()
            port_ret['year'] = year
            port_ret['portfolio'] = f'{eps_rank}_{lev_rank}'
            results.append(port_ret)

returns_df = pd.concat(results).reset_index(drop=True)